In [11]:
!pip install langchain==0.1.8
!pip install langchain-openai==0.0.6
!pip install python-dotenv==1.0.0

DEPRECATION: dropbox 11.36.0 has a non-standard dependency specifier stone>=2.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of dropbox or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: dropbox 11.36.0 has a non-standard dependency specifier stone>=2.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of dropbox or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: dropbox 11.36.0 has a non-standard dependency specifier stone>=2.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of dropbox or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import DirectoryLoader

In [5]:
load_dotenv()

True

In [44]:
#METADATA (provide info about the chatbot):
metadata = {
    "name": "harari",
    "sentiment": "optimistic", #can be "optimistic" or "pessimistic"
    "source": "https://hai.stanford.edu/news/historian-and-technologist-yuval-harari-and-fei-fei-li-discussion"
}

In [3]:
def write_to_file(filename, text):
    try:
        directory = os.path.dirname(filename)
        if not os.path.exists(directory):
            os.makedirs(directory)
        with open(filename, 'w') as file:
            file.write(text)
        print("Text successfully written to", filename)
    except Exception as e:
        print("An error occurred:", str(e))

In [6]:
llm = ChatOpenAI(model="gpt-4-turbo-preview",
                 temperature=0.7,
                 model_kwargs={
                    "frequency_penalty": 0.0,
                     "presence_penalty": 0.0,
                     "top_p": 1.0,
                  })

In [7]:
loader = DirectoryLoader('./source', glob="**/*.md")
raw_text = loader.load()

In [8]:
first_cycle_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", """
    Read the following transcript and extract all the arguments made about AI safety. Make sure they are self-contained.

  You must stick as close as possible to the transcript - use the author's own words and tone of voice.

  You must write each argument in a valid YAML format, surrounded with backticks.
  You must separate each argument with a new line.


  The simplest possible argument must at least contain three claims:

  ```
  claim: "Top-level claim"
  premises:
    - claim: "First independent premise supporting the top-level claim"
    - claim: "Second independent premise supporting the top-level claim"
  ```

  And here's an example of a more complex argument, which also includes examples to illustrate lower-level claims:

  ```
  claim: "Top-level claim"
  premises:
    - claim: "First independent premise supporting the top-level claim"
    - claim: "Supporting premise for the top-level claim"
      example: "Example supporting this premise"
    - claim: "Another supporting premise for the top-level claim"
    - claim: "Second independent premise supporting the top-level claim"
      premises:
        - claim: "Supporting premise for the second independent premise"
        - claim: "Another supporting premise for the second independent premise"
        - claim: "Independent premise supporting the second independent premise"
          example: "Example supporting this independent premise"
  ```

  Here's how to read this structure:

  The top-level claim is the main argument.
  Directly nested under the claim are independent premises. These provide justification independently of other premises.
  An example can be nested directly under a claim to provide further context or support.
  Just like the top-level claim, each premise can itself be supported by further individual premises, or examples, creating a nested structure.

  # Here is the transcript:

  {transcript}
  """)
])

In [9]:
first_cycle_chain = first_cycle_prompt | llm

In [11]:
first_cycle_response = first_cycle_chain.invoke({"transcript": raw_text[0].page_content})

In [12]:
first_cycle_response

AIMessage(content='```\nclaim: "AI has the potential to significantly benefit humanity."\npremises:\n  - claim: "AI can improve healthcare, making the best and cheapest healthcare in history available for billions on their cell phones."\n  - claim: "AI\'s integration with biology could lead to unprecedented advances in understanding and treating human conditions."\n```\n\n```\nclaim: "AI technology could lead to dystopian outcomes."\npremises:\n  - claim: "Liberal democracies could be undermined by machines that subtly control human lives."\n  - claim: "Totalitarian regimes could exploit AI to monitor and control every aspect of citizens\' lives, creating an omnipresent police state."\n```\n\n```\nclaim: "Human emotions, such as love, may be resistant to AI manipulation."\npremises:\n  - claim: "Despite AI\'s advances, it is still far from achieving true consciousness or experiencing feelings."\n  - claim: "The complexity and depth of human emotions like love present a significant chal

In [13]:
write_to_file("source/step_1/results.md", first_cycle_response.content)

Text successfully written to source/step_1/results.md


In [15]:
second_cycle_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", """
  Based on the following transcript, make the arguments clear and distinct. 
  You may need to merge similar arguments to create a better, more logical argument.

  Create the best, strongest possible version of the arguments, here's what to do:

  - Make sure the arguments is self-contained
  - Make the arguments understandable on their own, out-of-context
  - Remember, arguments are not a description or an explanation
  - Premise must always give a reason to believe the claim above
  - Avoid using pronouns in premises
  - A claim can have a maximum of two child claims (premises), rewrite if needed

  # Argument format 

  You must write each argument in valid YAML format, surrounded with backticks.
  Separate each argument with new line.
  You must stick as closely as possible to the transcript. 
  Above all, you must express the argument in the words of the author, stick as close as possible to the tone of voice and phrases used in the transcript.



  Here are some examples:

  A simple argument might look like this: 

  ```
  claim: "Top-level claim"
  premises:
    - claim: "First independent premise supporting the top-level claim"
    - claim: "Second independent premise supporting the top-level claim"
  ```

  And here's an example of a more complex argument, which also includes examples to illustrate lower-level claims:

  ```
  claim: "Top-level claim"
  premises:
    - claim: "First independent premise supporting the top-level claim"
    - claim: "Supporting premise for the top-level claim"
      example: "Example supporting this premise"
    - claim: "Another supporting premise for the top-level claim"
    - claim: "Second independent premise supporting the top-level claim"
      premises:
        - claim: "Supporting premise for the second independent premise"
        - claim: "Another supporting premise for the second independent premise"
        - claim: "Independent premise supporting the second independent premise"
          example: "Example supporting this independent premise"
  ```

  Here's how to read this structure:

  The top-level claim is the main argument.
  Directly nested under the claim are independent premises. These provide justification independently of other premises.
  An example can be nested directly under a claim to provide further context or support.
  Just like the top-level claim, each premise can itself be supported by further individual premises, or examples, creating a nested structure.


  # Arguments to improve:

  {all_arguments}

  # Transcript

  {transcript}
  """)
])

In [16]:
second_cycle_chain = second_cycle_prompt | llm

In [17]:
second_cycle_response = second_cycle_chain.invoke({"all_arguments": first_cycle_response.content, "transcript": raw_text[0].page_content})

In [18]:
second_cycle_response

AIMessage(content='```\nclaim: "AI has the potential to significantly benefit humanity."\npremises:\n  - claim: "AI can make the best and cheapest healthcare in history available for billions on their cell phones."\n  - claim: "AI\'s integration with biology could lead to unprecedented advances in understanding and treating human conditions."\n```\n\n```\nclaim: "AI technology could lead to dystopian outcomes."\npremises:\n  - claim: "Liberal democracies could be undermined by machines that exert subtle and pernicious control over human lives."\n  - claim: "Totalitarian regimes could use AI to create a police state able to monitor and control all aspects of citizens\' lives, 24 hours a day."\n```\n\n```\nclaim: "Human emotions, such as love, may be resistant to AI manipulation."\npremises:\n  - claim: "AI is still very far from achieving true consciousness or experiencing feelings."\n  - claim: "The complexity and depth of human emotions like love present a significant challenge to AI\

In [19]:
write_to_file("source/step_2/results.md", first_cycle_response.content)

Text successfully written to source/step_2/results.md


In [34]:
def isolate_arguments(args_combined):
    args_list = args_combined.split("\n```\n\n```\n")
    args_list[0] = args_list[0].split("```\n")[1]
    args_list[len(args_list) - 1] = args_list[len(args_list) - 1].split("\n```")[0]
    return args_list

In [36]:
isolated_arguments = isolate_arguments(second_cycle_response.content)

In [37]:
third_cycle_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world expert at creating accessible, persuasive explanations."),
    ("user", """
  Based on your own knowledge and the transcript, create a structured explanation for the following argument. Use the context from the transcript for the explanation.

  # Argument to use for the explanation

  {argument}


  The structured explanation must be directly based on the argument. You can also use the provided transcript for context.

  You must follow this YAML format:

  ```yaml
  counteragument_to: (what would be the argument, to which this argument is a counterargument? use your own knowledge. use bullet points)

  strongest_objection: (what is the strongest, good-faith, honest objection that a thoughful person might have? use bullet points)
  consequences_if_true: (if true, what would be the consequences? write in causal language,  use bullet points, max 3)

  link_to_ai_safety: (how is this linked to AI safety? 1 sentence.)

  simple_explanation: (explain this clearly to a college student in max. 4 sentences, speak persuasively as the author of this argument. don't use bullet points)

  examples: (max 3 examples, use bullet points)

  ```

  # Here is the transcript:

  {transcript}
  """)
])

In [38]:
third_cycle_chain = third_cycle_prompt | llm

In [75]:
def produce_final_args(args, explanations):
    final_args = []
    for i, explanation in enumerate(explanations):
        final_args.append(args[i] + "\n" + explanation.split("```yaml\n")[1])
    return final_args

In [76]:
def produce_explained_arguments(isolated_arguments):
    explained_args = []
    for argument in isolated_arguments:
        res = third_cycle_chain.invoke({"argument": argument, "transcript": raw_text[0].page_content})
        explained_args.append(res.content)
    return produce_final_args(isolated_arguments, explained_args)

In [77]:
final_arguments = produce_explained_arguments(isolated_arguments)

In [74]:
final_arguments

['claim: "AI has the potential to significantly benefit humanity."\npremises:\n  - claim: "AI can make the best and cheapest healthcare in history available for billions on their cell phones."\n  - claim: "AI\'s integration with biology could lead to unprecedented advances in understanding and treating human conditions."\ncounterargument_to:\n  - "AI poses too great a risk to humanity to pursue its development."\n  - "The negative consequences of AI outweigh its potential benefits."\n\nstrongest_objjection:\n  - "AI could be misused in ways that exacerbate inequality, violate privacy, or empower authoritarian regimes."\n\nconsequences_if_true:\n  - If AI can deliver the best and cheapest healthcare via cell phones, it could drastically reduce global health disparities.\n  - Integration of AI with biology could revolutionize our understanding and treatment of diseases, leading to longer, healthier lives.\n  - AI\'s potential benefits could spur global economic growth by creating new ind